In [ ]:
import pyxdf
import numpy as np
from sys import argv
from pathlib import Path

ModuleNotFoundError: No module named 'pyxdf'

In [ ]:
xdf_data_path = Path("xdf_data")

xdf_filepaths = list(xdf_data_path.rglob("*.xdf"))

xdf_data = {str(x).split("/")[1] : pyxdf.load_xdf(str(x)) for x in xdf_filepaths}

In [ ]:


def separate_trials_expmakers(exp_list):
    for i, string in enumerate(exp_list[1:]):
        if string == "task_block_start|BOX_BLOCK":
            return [exp_list[0:i+1], exp_list[i+1:]], False
        elif string == "task_block_start|JEBSEN_TAYLOR":
            return [exp_list[i+1:], exp_list[0:i+1]], True
    # If no empty string is found, return the whole list as the first trial, second is empty
    raise Exception(f"Separate exp: {exp_list}")

def separate_trials_latmarkers(lat_list):
    for i, string in enumerate(lat_list[1:]):
        if string == "repetition_start|2": # Do I need to do a similar logic like above?
            return lat_list[0:i+1], lat_list[i+1:]
    # If the marker is not found, return the whole list as first trial, second is empty
    raise Exception(f"Separate lat: {lat_list}")



In [ ]:
def collect_participant_trials(pnum):
    participant_trials = {
        "move_cans": {},
        "move_blocks": {}
    }

    exp_trial_jhft, exp_trial_blocks = None, None
    lat_trial_0, lat_trial_1 = None, None
    trials_swapped = False
    exp_markers_reached = False
    lat_markers_reached = False
    
    data, header = xdf_data[f"sub-{pnum:03}"]  # Assuming xdf_data is already loaded
    for stream in data:
        stream_series = stream["time_series"]
        stream_name = stream["info"]["name"][0]
        stream_series_list = list(stream_series)
        
        if stream_name == "LatencyMarkers":
            if lat_markers_reached:
                raise Exception("Duplication of latency markers!")
            lat_markers_reached = True
            
            stream_series_np = np.array(stream_series).flatten()
            lat_trial_0, lat_trial_1 = separate_trials_latmarkers(list(stream_series_np))
            if exp_markers_reached:
                participant_trials["move_cans"]["exp_markers"] = exp_trial_jhft
                participant_trials["move_blocks"]["exp_markers"] = exp_trial_blocks
                
                if trials_swapped:
                    participant_trials["move_cans"]["lat_markers"] = lat_trial_1
                    participant_trials["move_blocks"]["lat_markers"] = lat_trial_0
                else:
                    participant_trials["move_cans"]["lat_markers"] = lat_trial_0
                    participant_trials["move_blocks"]["lat_markers"] = lat_trial_1
        elif stream_name == "ExpMarkers" and not exp_markers_reached:
            exp_markers_reached = True
            
            stream_series_np = np.array(stream_series).flatten()
            # Not swapped can trial and then block trial
            [exp_trial_jhft, exp_trial_blocks], trials_swapped = separate_trials_expmakers(list(stream_series_np))
            
            if lat_markers_reached:
                participant_trials["move_cans"]["exp_markers"] = exp_trial_jhft
                participant_trials["move_blocks"]["exp_markers"] = exp_trial_blocks
                
                if trials_swapped:
                    participant_trials["move_cans"]["lat_markers"] = lat_trial_1
                    participant_trials["move_blocks"]["lat_markers"] = lat_trial_0
                else:
                    participant_trials["move_cans"]["lat_markers"] = lat_trial_0
                    participant_trials["move_blocks"]["lat_markers"] = lat_trial_1

    if not lat_markers_reached or not exp_markers_reached:
        raise Exception("Some of the required markers were not reached!")
    
    return participant_trials

In [ ]:
participants_trials = {}
for i in range(1, len(xdf_data) + 1):
    try:
        participants_trials[i] = collect_participant_trials(i)
    except Exception as e:
        ...

In [ ]:


def find_moved_blocks(p_trials):
    exp_markers = p_trials["move_blocks"]["exp_markers"]
    accepted_tasks = list(filter(lambda x: x.startswith("task_accept|"), exp_markers))
    moved_blocks = [int(x.split("task_accept|")[-1]) for x in accepted_tasks]
    return moved_blocks

def find_moved_blocks_latencies(p_trials):
    lat_markers = p_trials["move_blocks"]["lat_markers"]
    lat_applied = list(filter(lambda x: x.startswith("latency_applied"), lat_markers))[:5]
    latencies = [x.split("|")[1] for x in lat_applied]
    return latencies



In [ ]:


for i in participants_trials:
    # participant_moved_blocks = find_moved_blocks(participants_trials[i])
    # print(f"{i} : {len(participant_moved_blocks)} : {participant_moved_blocks}")
    latencies = find_moved_blocks_latencies(participants_trials[i])
    print(f"{i} : {latencies}")



In [ ]:


import pandas as pd



In [ ]:


pd.set_option("display.max_columns", None)     # show all columns
pd.set_option("display.max_rows", None)        # show all rows (optional)
pd.set_option("display.width", None)           # don't wrap columns
pd.set_option("display.max_colwidth", None)    # don't truncate cell content



In [ ]:


xlsx = pd.read_excel("questionnaire_data-561422-2025-11-07-1643.xlsx")



In [ ]:
xlsx

In [ ]:
xlsx_dict = {i : [] for i in range(1, 6)}
for xlsx_col in xlsx.columns:
    for i in range(1, 6):
        if str(xlsx_col).endswith(f"{i}"):
            xlsx_dict[i].append(xlsx_col)
            